<a href="https://colab.research.google.com/github/Ohoud-Almutairi/YOLOv8-KITTI-Object-Detection/blob/main/Autonomous_Driving_Object_Detection2_YoloV8_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Autonomous Driving Object Detection Using Convolutional Neural Networks (CNNs)

In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics

In [ ]:
import os
import zipfile
import requests

# Function to download and extract a dataset using requests
def download_and_extract(url, extract_to):
    # Get the filename from the URL
    zip_file_name = os.path.basename(url)
    zip_file_path = os.path.join(extract_to, zip_file_name)

    # Download the file using requests with streaming to handle large files
    print(f"Downloading {zip_file_name}...")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()  # Raise an exception for bad responses
        with open(zip_file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"Download complete.")

    # Extract the downloaded zip file
    print(f"Extracting {zip_file_name}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f'Dataset downloaded and extracted to {extract_to}')

    # Delete the ZIP file after extraction
    os.remove(zip_file_path)
    print(f'Deleted the ZIP file: {zip_file_path}')

# URLs for KITTI images and labels
kitti_image_url = 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip'
kitti_label_url = 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip'
extract_directory = './kitti_dataset'  # Specify your extraction directory

# Create directory if it doesn't exist
os.makedirs(extract_directory, exist_ok=True)

# Download and extract images and labels
download_and_extract(kitti_image_url, extract_directory)
download_and_extract(kitti_label_url, extract_directory)

In [ ]:
from pathlib import Path

# Set up paths for label and image directories, and classes file
label_dir = Path('/content/kitti_dataset/training/label_2')
image_2_dir = Path('/content/kitti_dataset/training/image_2')
use_dont_care = False  # Set to True to include 'DontCare' labels
classes_path = Path('classes.json')  # Ensure this file is available

# Define and create the output labels directory
OUT_LABELS_DIR = Path("/content/working/labels")
OUT_LABELS_DIR.mkdir(parents=True, exist_ok=True)

# Create a 'working' directory if needed
working_dir = Path('/content/working')
working_dir.mkdir(exist_ok=True)



preprocess the KITTI dataset labels

In [ ]:
import os
import csv
from PIL import Image

# Class labels
KEY_PEDESTRIAN = "Pedestrian"
KEY_CYCLIST = "Cyclist"
KEY_CAR = "Car"
KEY_VAN = "Van"
KEY_MISC = "Misc"
KEY_TRUCK = "Truck"
KEY_PERSON_SITTING = "Person_sitting"
KEY_TRAM = "Tram"
KEY_DONT_CARE = "DontCare"


# Class number mapping
CLAZZ_NUMBERS = {
    KEY_CAR: 0,
    KEY_PEDESTRIAN: 1,
    KEY_VAN: 2,
    KEY_CYCLIST: 3,
    KEY_TRUCK: 4,
    KEY_MISC: 5,
    KEY_TRAM: 6,
    KEY_PERSON_SITTING: 7,
    KEY_DONT_CARE: 8
}



def getSampleId(path):
    basename = os.path.basename(path)
    return os.path.splitext(basename)[0]

def resolveClazzNumberOrNone(clazz, use_dont_care):

    if use_dont_care and clazz == KEY_DONT_CARE:
        return CLAZZ_NUMBERS[clazz]
    elif clazz != KEY_DONT_CARE:
        return CLAZZ_NUMBERS[clazz]
    return None

def convertToYoloBBox(bbox, size):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (bbox[0] + bbox[1]) / 2.0
    y = (bbox[2] + bbox[3]) / 2.0
    w = bbox[1] - bbox[0]
    h = bbox[3] - bbox[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def readRealImageSize(img_path):
    return Image.open(img_path).size

def parseSample(lbl_path, img_path, use_dont_care):
    with open(lbl_path) as csv_file:
        reader = csv.DictReader(csv_file, fieldnames=["type", "truncated", "occluded", "alpha", "bbox2_left", "bbox2_top", "bbox2_right", "bbox2_bottom", "bbox3_height", "bbox3_width", "bbox3_length", "bbox3_x", "bbox3_y", "bbox3_z", "bbox3_yaw", "score"], delimiter=" ")
        yolo_labels = []
        for row in reader:
            clazz_number = resolveClazzNumberOrNone(row["type"], use_dont_care)
            if clazz_number is not None:
                size = readRealImageSize(img_path)
                bbox = (
                    float(row["bbox2_left"]),
                    float(row["bbox2_right"]),
                    float(row["bbox2_top"]),
                    float(row["bbox2_bottom"])
                )
                yolo_bbox = convertToYoloBBox(bbox, size)
                yolo_label = (clazz_number,) + yolo_bbox
                yolo_labels.append(yolo_label)
    return yolo_labels




print("Generating YOLO labels...")
sample_img_paths = []
for dir_path, sub_dirs, files in os.walk(label_dir):
    for file_name in files:
        if file_name.endswith(".txt"):
            lbl_path = os.path.join(dir_path, file_name)
            sample_id = getSampleId(lbl_path)
            img_path = os.path.join(image_2_dir, "{}.png".format(sample_id))
            sample_img_paths.append(img_path)
            yolo_labels = parseSample(lbl_path, img_path, use_dont_care)
            with open(os.path.join(str(OUT_LABELS_DIR), "{}.txt".format(sample_id)), "w") as yolo_label_file:
                for lbl in yolo_labels:
                    yolo_label_file.write("{} {} {} {} {}\n".format(*lbl))

print("YOLO labels generation complete!")




3. Prepare the Dataset
Prepare the images and labels for training.

In [ ]:
import os
import json
import shutil
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Class mapping
classes = {
    "Car": 0,
    "Pedestrian": 1,
    "Van": 2,
    "Cyclist": 3,
    "Truck": 4,
    "Misc": 5,
    "Tram": 6,
    "Person_sitting": 7
}

# Save classes to a JSON file
classes_path = 'classes.json'
with open(classes_path, 'w') as f:
    json.dump(classes, f, indent=4)

# Prepare images and labels
ims = sorted(list(image_2_dir.glob('*')))
labels = sorted(list(OUT_LABELS_DIR.glob('*')))
pairs = list(zip(ims, labels))

data_size = len(pairs)
# Check if the dataset is large enough
if data_size > 0:

    train, test = train_test_split(pairs, test_size=0.1, shuffle=True)
    # Create train and valid directories
    train_path = Path('/content/working/train').resolve()
    valid_path = Path('/content/working/valid').resolve()

    # Delete the directories if they exist
    if train_path.exists() and train_path.is_dir():
        shutil.rmtree(train_path)

    if valid_path.exists() and valid_path.is_dir():
        shutil.rmtree(valid_path)

    train_path.mkdir(exist_ok=True)
    valid_path.mkdir(exist_ok=True)

    # Copy training data
    for t_img, t_lb in tqdm(train):
        shutil.copy(t_img, train_path / t_img.name)
        shutil.copy(t_lb, train_path / t_lb.name)

    # Copy validation data
    for t_img, t_lb in tqdm(test):
        shutil.copy(t_img, valid_path / t_img.name)
        shutil.copy(t_lb, valid_path / t_lb.name)

print("\nData preparation complete!")


4. Create the YAML Configuration File
You need a YAML file to define the dataset structure.

In [ ]:
yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(train_path)}\nval: {str(valid_path)}'
with open('/content/working/kitti.yaml', 'w') as f:
    f.write(yaml_file)

5. Train the YOLOv8 Model
Now you can set up and train the YOLOv8 model.

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')


# Train the model
train_results = model.train(
    data='/content/working/kitti.yaml',  # Ensure the correct path to your YAML file
    epochs=15,
    batch=100,
    patience=3,
    mixup=0.1,
    project='yolov8n-kitti',  # Name of the project folder for saving results

)


6. Validate and Visualize Results
You can validate the model and visualize the training results.

In [ ]:
# Validate the model
valid_results = model.val()


In [ ]:

# Plot training results
import matplotlib.pyplot as plt
from PIL import Image



plt.figure(figsize=(10,20))
plt.imshow(Image.open('/content/yolov8n-kitti/train/results.png'))
plt.axis('off')
plt.show()



In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open('/content/yolov8n-kitti/train2/confusion_matrix.png'))
plt.axis('off')
plt.show()

7. Make Predictions
You can make predictions on a set of images.

In [ ]:
# Make predictions
preds = model.predict([test[idx][0] for idx in np.random.randint(0,len(test),(20,))],save=True)


In [ ]:
preds = list(Path('yolov8n-kitti/train3').glob('*'))

# Function to plot images
def plot_images(images):
    num_images = len(images)
    rows = num_images
    cols = 1
    fig, axes = plt.subplots(rows, cols, figsize=(15, 80))
    for ax in axes.flat:
        ax.axis('off')
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        axes[i].imshow(img)

    plt.tight_layout()
    plt.show()

# Plot predicted images
plot_images(preds)

In [ ]:
# Example of calculating accuracy
def calculate_accuracy(predictions, ground_truth):
    correct = 0
    total = len(predictions)

    for pred, true_label in zip(predictions, ground_truth):
        if pred == true_label:
            correct += 1

    accuracy = correct / total
    return accuracy

# Example ground truth labels and model predictions
ground_truth_labels = [0, 1, 1, 0, 1]
model_predictions = [0, 1, 0, 0, 1]

# Calculate accuracy
accuracy = calculate_accuracy(model_predictions, ground_truth_labels)
print(f"Accuracy: {accuracy}")